In [1]:
! pip install numpy
! pip install tensorflow
# ! pip install string


In [3]:
import numpy as np
import string
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Lambda, Conv1D
from tensorflow.keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))
# Model architecture
model = Sequential()

model.add(Embedding(input_dim=28, output_dim=32, input_length=30))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(LSTM(192))
model.add(Dense(64, activation='tanh'))
model.add(Dense(26, activation='softmax'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.8,beta_2=0.6)
model.compile(optimizer=optimizer, loss=root_mean_squared_error, metrics=['accuracy'])

2023-07-25 00:51:39.237635: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-25 00:51:39.264220: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-25 00:51:39.265157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 00:51:39.864327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-25 00:51:40.460938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

In [4]:
model.load_weights('HangmanMode.h5')

In [5]:
import string

In [6]:
all_characters = ['_','&'] + list(string.ascii_lowercase)
char_to_int = {char: idx for idx, char in enumerate(all_characters)}

int_to_char = {idx: char for char, idx in char_to_int.items()}

In [7]:
def word2vec(word):
    word = list(word)
    word = [[char for char in wd] for wd in word]
    word_int = [[char_to_int[char] for char in word] for word in word]
    word_padded = [array+[1]*(30-len(array)) for array in word_int]
    return np.array(word_padded)

In [28]:
wordtest = word2vec(["elephant"])

In [29]:
wordtest.shape

(1, 30)

In [30]:
y = model.predict(wordtest)

1/1 [==============================] - 0s 16ms/step


In [31]:
k = np.argsort(y)

In [32]:
k.shape

(1, 26)

In [26]:
k[0][i]

21

In [35]:
for i in range(k.shape[1]):
    print(int_to_char[k[0][i]])

x
t
v
o
f
m
u
e
k
d
h
l
_
b
w
s
&
c
j
i
n
g
a
r
p
q


In [6]:
dict = (open("/home/divyansh/temp/Hangman Game/training_words.txt",'r')).read()
dict = dict.splitlines()

In [11]:
dict[89]

'vitaceous'

In [15]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define your word dictionary with 2.5 lac words
word_dictionary = dict

# Hyperparameters
EPISODES = 100
MAX_ATTEMPTS = 6
MAX_WORD_LENGTH = 20
LEARNING_RATE = 0.001
GAMMA = 0.9
EPSILON = 0.2

# Function to randomly sample a word for hangman
def get_random_word():
    return random.choice(word_dictionary)

# Function to mask the word
def mask_word(word, guessed_letters):
    return "".join([c if c in guessed_letters else "_" for c in word])

# Define the RL agent using Deep Q Networks (DQN)
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        self.model = self.build_model()
    
    def build_model(self):
        model = tf.keras.Sequential()
        model.add(layers.Embedding(input_dim=26, output_dim=10, input_length=self.state_size))
        model.add(layers.Flatten())
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=self.optimizer)
        return model
    
    def act(self, state):
        if np.random.rand() <= EPSILON:
            return random.choice(range(self.action_size))
        return np.argmax(self.model.predict(state)[0])

    def train(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = (reward + GAMMA * np.amax(self.model.predict(next_state)[0]))
        target_f = self.model.predict(state)
        target_f[0][action] = target
        self.model.fit(state, target_f, epochs=1, verbose=1)

def play_hangman(agent):
    action_size = 26

    for episode in range(EPISODES):
        word = get_random_word()
        print("word is: ",word)
        guessed_letters = set()
        state = np.array([ord(c) - ord('a') for c in word])
        state = np.expand_dims(state, axis=0)

        for attempt in range(MAX_ATTEMPTS):
            action = agent.act(pad_sequences(state, maxlen=MAX_WORD_LENGTH, padding='post'))
            guessed_letter = chr(action + ord('a'))
            guessed_letters.add(guessed_letter)
            
            if guessed_letter in word:
                reward = 1.0
            else:
                reward = -0.5
            
            next_word = mask_word(word, guessed_letters)
            next_state = np.array([ord(c) - ord('a') for c in next_word])
            next_state = np.expand_dims(next_state, axis=0)
            next_state = pad_sequences(next_state, maxlen=MAX_WORD_LENGTH, padding='post')
            
            done = "_" not in next_word
            agent.train(state, action, reward, next_state, done)

            state = next_state

        if episode % 100 == 0:
            print(f"Episode: {episode}, Success rate: {evaluate(agent)}")

def evaluate(agent, num_trials=1000):
    success_count = 0

    for _ in range(num_trials):
        word = get_random_word()
        guessed_letters = set()

        for _ in range(MAX_ATTEMPTS):
            state = np.array([ord(c) - ord('a') for c in word])
            state = np.expand_dims(state, axis=0)
            state = pad_sequences(state, maxlen=MAX_WORD_LENGTH, padding='post')
            
            action = agent.act(state)
            guessed_letter = chr(action + ord('a'))
            guessed_letters.add(guessed_letter)

            if guessed_letter in word:
                break

        if "_" not in mask_word(word, guessed_letters):
            success_count += 1

    return success_count / num_trials


In [16]:
# Main function
if __name__ == "__main__":
    action_size = 26

    agent = DQNAgent(30, action_size)
    play_hangman(agent)

word is:  supawn


ValueError: in user code:

    File "/home/divyansh/temp/Hangman Game/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/home/divyansh/temp/Hangman Game/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/divyansh/temp/Hangman Game/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/home/divyansh/temp/Hangman Game/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/home/divyansh/temp/Hangman Game/.venv/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/divyans